# Runs Reddit API to collect posts containing a search term, and returns post with comments on it

### Import the Reddit credentials and modules required

In [ ]:
from redditcredentials import CLIENT_ID, CLIENT_SECRET, USERNAME_REDDIT, PASSWORD_REDDIT
import requests

### Setup the request for Reddit API, OAuth authentication

In [ ]:
# Authorisation code from https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c, as tried what API github suggested and did not work
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)

data = {'grant_type': 'password',
        'username': USERNAME_REDDIT,
        'password': PASSWORD_REDDIT}

headers = {'User-Agent': 'TopicSentimentAcrossSN/0.0.1'}

res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

TOKEN = res.json()['access_token']

headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

### Set search term and create the dictionary to hold posts

In [ ]:
search_term = 'Scotrail'
reddit_posts = {}

### Function to get the comments from the posts

In [ ]:
def getCommentsFromPosts(post_id, post_title):
        comment_results = requests.get("https://oauth.reddit.com/r/all/comments/" + str(post_id),
                                headers=headers, params={'limit': '25'})

        print("PRINTING COMMENT RESULTS FOR THE TOPIC: \n")

        comments = comment_results.json()[1]['data']['children'][:-1]   #slices last value as it is just list of id's
        all_comments = []

        for comment in comments:
                comment_text = comment['data']['body']
                all_comments += [comment_text]
        
        reddit_posts[post_title] = all_comments

### Find results on search term from reddit posts, and collect comments containing that topic

In [ ]:
results = requests.get("https://oauth.reddit.com/r/all/search", 
                headers=headers, params={'limit': '25', 'q':{search_term}})

print("PRINTING SEARCH RESULTS FOR THE TOPIC: \n")

posts = results.json()['data']['children']

for post in posts:
        post_title = post['data']['title']
        print(post_title)
        post_id = post['data']['id']
        getCommentsFromPosts(post_id, post_title)

### Dictionary results

In [ ]:
print(reddit_posts)

## VADER Analysis test

In [ ]:
!pip install vaderSentiment

In [ ]:
reddit_results = []
for post in reddit_posts.values():
   # print(post)
    reddit_results.extend(post)

print(reddit_results)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# Example code:

f = open("reddit_scores.txt", "x", encoding="utf-8")

for reddit_post in reddit_results:
  vs = analyzer.polarity_scores(reddit_post)
  compound = vs['compound']
  sentiment_score = compound * 100
  print("The post '" + reddit_post + "' has a sentiment score of: " + (str(sentiment_score)) + "%")
  
  if (compound >= 0.05):
    print("The post was POSITIVE")
    f.write("\n" + reddit_post + ": RESULT = POSITIVE")
  elif (compound <= -0.05):
    print("The post was NEGATIVE")
    f.write("\n" + reddit_post + ": RESULT = NEGATIVE")
  else:
    print("The post was NEUTRAL")
    f.write("\n" + reddit_post + ": RESULT = NEUTRAL")
  print("{:-<65} {}".format(reddit_post, str(vs)))

f.close()